# Assignment 3

# Instructions

1. You have to use only this notebook for all your code.
2. All the results and plots should be mentioned in this notebook.
3. For final submission, submit this notebook along with the report ( usual 2-4 pages, latex typeset, which includes the challenges faces and details of additional steps, if any)
4. Marking scheme
    -  **60%**: Your code should be able to detect bounding boxes using resnet 18, correct data loading and preprocessing. Plot any 5 correct and 5 incorrect sample detections from the test set in this notebook for both the approached (1 layer and 2 layer detection), so total of 20 plots.
    -  **20%**: Use two layers (multi-scale feature maps) to detect objects independently as in SSD (https://arxiv.org/abs/1512.02325).  In this method, 1st detection will be through the last layer of Resnet18 and the 2nd detection could be through any layer before the last layer. SSD uses lower resolution layers to detect larger scale objects. 
    -  **20%**: Implement Non-maximum suppression (NMS) (should not be imported from any library) on the candidate bounding boxes.
    
5. Report AP for each of the three class and mAP score for the complete test set.

In [2]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion()
# Import other modules if required
import os
from scipy.misc import imread
from matplotlib import pyplot as plt
from skimage import transform
# Can use other libraries as well

resnet_input = 224 #size of resnet18 input images

In [53]:
# Choose your hyper-parameters using validation data
batch_size = 2
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
Use the following links to locally download the data:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>The dataset consists of images from 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, i.e. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the three classes(aeroplane, bottle, chair). For parsing the xml file, you can import xml.etree.ElementTree for you. <br/>
<br/> Organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be four. This is important for applying the sliding window method later.


In [4]:
classes = ('__background__',
           'aeroplane',
           'bottle','chair'
           )


/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/006916.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/008301.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/003100.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/005026.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/002565.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/006635.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/006784.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/001269.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/001907.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/007448.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/007486.xml
407 141
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/008068.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/009268.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/007642.xml
VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/006235.xml
VOCtrainval_06-No

[0. 0. 0. 1.]


In [6]:
import xml.etree.ElementTree as ET
from glob import glob
images = []
categs = []
def build_dataset(data_path):
    data = glob(os.path.join(data_path,"*","*","*","*.jpg"))
    count = 0
    for file in data:
        image = imread(file, mode="RGB")
        img_name = file.split('/')[-1].split('.')[0]
        annotation_path = data_path +'/VOCdevkit/VOC2007/Annotations/'+img_name + '.xml'
        print(annotation_path)
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        for child in root:
#             print(child.tag)
            if child.tag == 'object':
                if child[0].tag == 'name':
                    if child[0].text in classes:
                        xmin = int(child[4][0].text)
                        ymin = int(child[4][1].text)
                        xmax = int(child[4][2].text)
                        ymax = int(child[4][3].text)
                        cropped = image[xmin:xmax,ymin:ymax]
#                         cropped=transform.resize(cropped, (224,224),anti_aliasing=True)
                        images.append(cropped)
                        categ = np.zeros(4)
                        if child[0].text == classes[0]:
                            categ[0] = 1
                        elif child[0].text == classes[1]:
                            categ[1] = 1
                        elif child[0].text == classes[2]:
                            categ[2] = 1
                        elif child[0].text == classes[3]:
                            categ[3] = 1
                        categs.append(categ)
                        print(xmax-xmin,ymax-ymin)

                            
        count += 1
#         print(count)
        if count == 50:
            break
build_dataset('VOCtrainval_06-Nov-2007')
plt.imshow(images[9])
plt.show()
print(categs[9])

In [ ]:
class voc_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        
    def __len__(self):
        # Begin
        
    def __getitem__(self, idx):
       # Begin
    

## Train the netwok
<br/>You can train the network on the created dataset. This will yield a classification network on the 4 classes of the VOC dataset. 

In [ ]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.ToTensor(),
                                         transforms.RandomHorizontalFlip()])
train_dataset = voc_dataset(root_dir='', train=True, transform=composed_transform) # Supply proper root_dir
test_dataset = voc_dataset(root_dir='', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Fine-tuning
Use the pre-trained network to fine-tune the network in the following section:

In [ ]:
resnet18 = models.resnet18(pretrained=True)

resnet18.fc = nn.Linear(resnet18.fc.in_features, 4)

# Add code for using CUDA here

In [ ]:
criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = optim.SGD(resnet18.parameters(), learning_rate, hyp_momentum)

In [ ]:
#One Layer Detection
def train():
    # Begin

In [ ]:
%time train()

In [ ]:
#Two Layer Detection (SSD)
def train():
    # Begin

In [ ]:
%time train()

# Testing and Accuracy Calculation
For applying detection, use a slding window method to test the above trained trained network on the detection task:<br/>
Take some windows of varying size and aspect ratios and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value. There is a similar approach used in the paper -Faster RCNN by Ross Girshick, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide. You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [ ]:
def sliding_window():
    # Begin

Apply non_maximum_supression to reduce the number of boxes. You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [ ]:
def non_maximum_supression(boxes,threshold = 0.3):
    # 

Test the trained model on the test dataset.

In [ ]:
#One Layer Detection
def test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model

In [ ]:
%time test(resnet18)

In [ ]:
#Two Layer Detection
def test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model

In [ ]:
%time test(resnet18)